In [37]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests

import folium
from folium import plugins
from folium.plugins import HeatMap


# Import API key
#from api_keys import weather_api_key

In [38]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
crashes_df = pd.read_csv("data/nyc_mv_crash_data_truncated4.csv")


# Display sample data
crashes_df

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,9/11/2021,2:39,NaN,NaN,NaN,2.0,0.0,Sedan,Sedan,NaN,NaN,NaN
1,3/26/2022,11:45,NaN,NaN,NaN,1.0,0.0,Sedan,NaN,NaN,NaN,NaN
2,6/29/2022,6:55,NaN,NaN,NaN,0.0,0.0,Sedan,Pick-up Truck,NaN,NaN,NaN
3,9/11/2021,9:35,BROOKLYN,40.667202,-73.866500,0.0,0.0,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,40.683304,-73.917274,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,9/8/2016,14:00,BRONX,40.820297,-73.854645,0.0,0.0,Sedan,Sedan,NaN,NaN,NaN
1048571,8/23/2016,16:55,NaN,40.692722,-73.968750,0.0,0.0,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1048572,9/11/2016,14:45,NaN,40.733430,-73.868890,0.0,0.0,Sedan,Sedan,NaN,NaN,NaN
1048573,9/9/2016,17:00,NaN,40.770065,-73.959274,0.0,0.0,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


In [39]:
cols_to_check = ['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE', 'LONGITUDE', 'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']

crashes_df.dropna(subset=cols_to_check, inplace=True)

crashes_df

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
3,9/11/2021,9:35,BROOKLYN,40.667202,-73.866500,0.0,0.0,Sedan,NaN,NaN,NaN,NaN
4,12/14/2021,8:13,BROOKLYN,40.683304,-73.917274,0.0,0.0,NaN,NaN,NaN,NaN,NaN
7,12/14/2021,8:17,BRONX,40.868160,-73.831480,2.0,0.0,Sedan,Sedan,NaN,NaN,NaN
8,12/14/2021,21:10,BROOKLYN,40.671720,-73.897100,0.0,0.0,Sedan,NaN,NaN,NaN,NaN
9,12/14/2021,14:58,MANHATTAN,40.751440,-73.973970,0.0,0.0,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1048563,9/10/2016,18:39,MANHATTAN,40.746490,-74.001335,0.0,0.0,Station Wagon/Sport Utility Vehicle,Sedan,Sedan,NaN,NaN
1048564,8/22/2016,15:30,QUEENS,40.665504,-73.856120,0.0,0.0,Convertible,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1048566,8/29/2016,13:45,QUEENS,40.686516,-73.846540,0.0,0.0,Sedan,NaN,NaN,NaN,NaN
1048568,9/10/2016,11:00,QUEENS,40.680490,-73.774704,1.0,0.0,Station Wagon/Sport Utility Vehicle,Bike,NaN,NaN,NaN


In [40]:
crashes_df['NUMBER OF VEHICLES'] = crashes_df[['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5']].count(axis=1)
crashes_df.drop(columns=['NUMBER OF PERSONS INJURED', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'], inplace=True)
crashes_df.head(500)

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS KILLED,NUMBER OF VEHICLES
3,9/11/2021,9:35,BROOKLYN,40.667202,-73.866500,0.0,1
4,12/14/2021,8:13,BROOKLYN,40.683304,-73.917274,0.0,0
7,12/14/2021,8:17,BRONX,40.868160,-73.831480,0.0,2
8,12/14/2021,21:10,BROOKLYN,40.671720,-73.897100,0.0,1
9,12/14/2021,14:58,MANHATTAN,40.751440,-73.973970,0.0,2
...,...,...,...,...,...,...,...
784,4/16/2021,15:45,BROOKLYN,40.731422,-73.946410,0.0,2
785,4/15/2021,8:20,MANHATTAN,40.761234,-73.963890,0.0,1
786,4/10/2021,19:09,BROOKLYN,40.668976,-73.906680,0.0,2
788,4/15/2021,18:20,BROOKLYN,40.637276,-73.931770,0.0,2


In [41]:
fatality_df = crashes_df[crashes_df['NUMBER OF PERSONS KILLED']>0]

fatality_df

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS KILLED,NUMBER OF VEHICLES
591,4/15/2021,15:18,BROOKLYN,40.620487,-74.029305,1.0,1
1350,7/8/2021,22:03,MANHATTAN,40.721474,-73.983830,1.0,2
2346,8/27/2021,9:15,MANHATTAN,40.805740,-73.942764,1.0,1
2437,9/11/2021,18:18,BROOKLYN,40.684204,-73.968060,1.0,2
2607,4/8/2021,19:55,BRONX,40.830307,-73.898730,1.0,1
...,...,...,...,...,...,...,...
1031260,10/7/2016,14:47,MANHATTAN,0.000000,0.000000,1.0,1
1032436,10/4/2016,9:49,MANHATTAN,40.719933,-73.978760,1.0,1
1035830,9/30/2016,12:16,STATEN ISLAND,40.626663,-74.075650,1.0,1
1037943,9/13/2016,19:41,MANHATTAN,40.810684,-73.954520,1.0,1


In [42]:
brooklyn_df = fatality_df[fatality_df['BOROUGH'] == 'BROOKLYN']
brooklyn_df

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS KILLED,NUMBER OF VEHICLES
591,4/15/2021,15:18,BROOKLYN,40.620487,-74.029305,1.0,1
2437,9/11/2021,18:18,BROOKLYN,40.684204,-73.968060,1.0,2
5831,4/24/2021,3:43,BROOKLYN,40.643063,-73.951660,1.0,2
7075,8/24/2021,21:23,BROOKLYN,40.675716,-73.914200,1.0,1
8480,4/19/2021,8:39,BROOKLYN,40.614240,-73.988240,1.0,1
...,...,...,...,...,...,...,...
983864,12/9/2016,8:30,BROOKLYN,40.679466,-73.900370,1.0,4
984445,12/19/2016,22:21,BROOKLYN,40.602688,-73.933556,1.0,1
986721,11/19/2016,15:45,BROOKLYN,40.591106,-73.965130,1.0,2
997656,11/13/2016,17:15,BROOKLYN,40.684147,-73.922966,1.0,0


In [43]:
bronx_df = fatality_df[fatality_df['BOROUGH'] == 'BRONX']
bronx_df

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS KILLED,NUMBER OF VEHICLES
2607,4/8/2021,19:55,BRONX,40.830307,-73.898730,1.0,1
4137,12/16/2021,0:15,BRONX,40.849550,-73.853050,2.0,1
15476,6/29/2022,23:54,BRONX,40.814266,-73.912964,1.0,2
24793,10/27/2021,20:22,BRONX,40.861404,-73.870590,1.0,1
29258,10/2/2021,2:46,BRONX,40.844970,-73.902470,1.0,1
...,...,...,...,...,...,...,...
995716,11/8/2016,16:00,BRONX,40.809105,-73.922900,1.0,1
1000350,11/21/2016,17:35,BRONX,40.866047,-73.882744,1.0,1
1014606,10/18/2016,16:20,BRONX,40.836346,-73.852220,1.0,1
1016569,10/10/2016,21:36,BRONX,40.825200,-73.914760,1.0,0


In [44]:
manhattan_df = fatality_df[fatality_df['BOROUGH'] == 'MANHATTAN']
manhattan_df

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS KILLED,NUMBER OF VEHICLES
1350,7/8/2021,22:03,MANHATTAN,40.721474,-73.983830,1.0,2
2346,8/27/2021,9:15,MANHATTAN,40.805740,-73.942764,1.0,1
2906,4/19/2021,19:13,MANHATTAN,40.822890,-73.942080,1.0,1
10883,3/30/2021,13:09,MANHATTAN,40.774370,-73.963554,1.0,1
11450,9/19/2021,23:17,MANHATTAN,40.751892,-73.967600,1.0,2
...,...,...,...,...,...,...,...
1027224,9/17/2016,10:41,MANHATTAN,40.769060,-73.961105,1.0,1
1027799,9/28/2016,19:40,MANHATTAN,40.797306,-73.934460,1.0,1
1031260,10/7/2016,14:47,MANHATTAN,0.000000,0.000000,1.0,1
1032436,10/4/2016,9:49,MANHATTAN,40.719933,-73.978760,1.0,1


In [45]:
staten_df = fatality_df[fatality_df['BOROUGH'] == 'STATEN ISLAND']
staten_df

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS KILLED,NUMBER OF VEHICLES
3042,12/17/2021,23:50,STATEN ISLAND,40.547924,-74.166954,1.0,1
19612,5/6/2021,14:02,STATEN ISLAND,40.505527,-74.238190,1.0,2
70165,7/29/2021,10:25,STATEN ISLAND,40.618160,-74.084920,1.0,1
118113,7/10/2022,20:47,STATEN ISLAND,40.508614,-74.223750,3.0,2
133211,5/26/2022,7:05,STATEN ISLAND,40.613373,-74.063255,1.0,1
166865,3/5/2021,6:38,STATEN ISLAND,40.593250,-74.110650,1.0,1
170679,1/13/2021,13:05,STATEN ISLAND,40.626663,-74.075650,1.0,1
179535,1/29/2021,7:46,STATEN ISLAND,40.603615,-74.069310,1.0,1
196262,11/30/2020,22:06,STATEN ISLAND,40.566494,-74.113770,1.0,1
202408,10/15/2020,9:46,STATEN ISLAND,40.590750,-74.088370,1.0,1


In [46]:
queens_df = fatality_df[fatality_df['BOROUGH'] == 'QUEENS']
queens_df

,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS KILLED,NUMBER OF VEHICLES
4456,4/21/2021,0:55,QUEENS,40.692135,-73.834850,1.0,3
7073,8/21/2021,8:07,QUEENS,40.758125,-73.928505,1.0,2
10896,5/3/2021,23:10,QUEENS,40.687750,-73.790390,1.0,3
15136,11/17/2021,6:00,QUEENS,40.666750,-73.779945,1.0,1
24795,12/13/2021,16:52,QUEENS,40.710400,-73.866630,1.0,1
...,...,...,...,...,...,...,...
987502,11/13/2016,10:32,QUEENS,40.740036,-73.702420,1.0,1
990510,11/18/2016,12:39,QUEENS,40.773594,-73.911790,1.0,1
1022717,9/27/2016,6:20,QUEENS,40.676968,-73.835270,1.0,1
1025542,9/19/2016,6:31,QUEENS,40.663620,-73.777176,1.0,1


In [47]:
fatality_df['DATETIME'] = pd.to_datetime(fatality_df['CRASH DATE'])
fatality_df['DATETIME'] = (fatality_df['DATETIME'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

fatality_df

C:\Users\sbenn\AppData\Local\Temp\ipykernel_5900\447860299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fatality_df['DATETIME'] = pd.to_datetime(fatality_df['CRASH DATE'])
C:\Users\sbenn\AppData\Local\Temp\ipykernel_5900\447860299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fatality_df['DATETIME'] = (fatality_df['DATETIME'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')


,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,NUMBER OF PERSONS KILLED,NUMBER OF VEHICLES,DATETIME
591,4/15/2021,15:18,BROOKLYN,40.620487,-74.029305,1.0,1,1618444800
1350,7/8/2021,22:03,MANHATTAN,40.721474,-73.983830,1.0,2,1625702400
2346,8/27/2021,9:15,MANHATTAN,40.805740,-73.942764,1.0,1,1630022400
2437,9/11/2021,18:18,BROOKLYN,40.684204,-73.968060,1.0,2,1631318400
2607,4/8/2021,19:55,BRONX,40.830307,-73.898730,1.0,1,1617840000
...,...,...,...,...,...,...,...,...
1031260,10/7/2016,14:47,MANHATTAN,0.000000,0.000000,1.0,1,1475798400
1032436,10/4/2016,9:49,MANHATTAN,40.719933,-73.978760,1.0,1,1475539200
1035830,9/30/2016,12:16,STATEN ISLAND,40.626663,-74.075650,1.0,1,1475193600
1037943,9/13/2016,19:41,MANHATTAN,40.810684,-73.954520,1.0,1,1473724800


In [48]:
%%capture --no-display

# Configure the map plot
map_plot_2 = fatality_df.hvplot.points(
    "LONGITUDE",
    "LATITUDE",
    geo = True,
    hover_cols=['LATITUDE', 'LONGITUDE', 'BOROUGH'],
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    xlim = (-74.2, -73.5),
    ylim = (40.5, 41),
    color = "BOROUGH"
)
# Display the map
#hvplot.save(map_plot_2, 'map.html')
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [LONGITUDE,LATITUDE]   (BOROUGH)

In [49]:
# Create a new map centered on New York City
nyc_coords = (40.7128, -74.0060)
f = folium.Figure(width=1000, height=500)
crash_map = folium.Map(location= nyc_coords, tiles="openstreetmap", zoom_start=10).add_to(f)
#crash_map = folium.Map(location=nyc_coords, zoom_start=11)

# Add a heatmap layer using the latitude and longitude columns from your DataFrame
heat_data = [[row['LATITUDE'], row['LONGITUDE']] for index, row in fatality_df.iterrows()]
HeatMap(heat_data).add_to(crash_map)

# Display the map
crash_map

In [50]:
# Create a new map centered on New York City
nyc_coords = (40.67,-73.94)
f = folium.Figure(width=1200, height=800)
crash_map = folium.Map(location=nyc_coords, tiles="openstreetmap", zoom_start=12).add_to(f)
#crash_map = folium.Map(location=nyc_coords, zoom_start=11)

# Add a heatmap layer using the latitude and longitude columns from your DataFrame
heat_data = [[row['LATITUDE'], row['LONGITUDE']] for index, row in brooklyn_df.iterrows()]
HeatMap(heat_data).add_to(crash_map)

# Display the map
crash_map

In [51]:
# Create a new map centered on New York City
nyc_coords = (40.84,-73.86)
f = folium.Figure(width=1200, height=800)
crash_map = folium.Map(location= nyc_coords, tiles="openstreetmap", zoom_start=13).add_to(f)
#crash_map = folium.Map(location=nyc_coords, zoom_start=11)

# Add a heatmap layer using the latitude and longitude columns from your DataFrame
heat_data = [[row['LATITUDE'], row['LONGITUDE']] for index, row in bronx_df.iterrows()]
HeatMap(heat_data).add_to(crash_map)

# Display the map
crash_map

In [52]:
# Create a new map centered on New York City
nyc_coords = (40.77, -73.97)
f = folium.Figure(width=1200, height=800)
crash_map = folium.Map(location= nyc_coords, tiles="openstreetmap", zoom_start=12).add_to(f)
#crash_map = folium.Map(location=nyc_coords, zoom_start=11)

# Add a heatmap layer using the latitude and longitude columns from your DataFrame
heat_data = [[row['LATITUDE'], row['LONGITUDE']] for index, row in manhattan_df.iterrows()]
HeatMap(heat_data).add_to(crash_map)

# Display the map
crash_map

In [53]:
# Create a new map centered on New York City
nyc_coords = (40.713, -73.813)
f = folium.Figure(width=1200, height=800)
crash_map = folium.Map(location= nyc_coords, tiles="openstreetmap", zoom_start=12).add_to(f)
#crash_map = folium.Map(location=nyc_coords, zoom_start=11)

# Add a heatmap layer using the latitude and longitude columns from your DataFrame
heat_data = [[row['LATITUDE'], row['LONGITUDE']] for index, row in queens_df.iterrows()]
HeatMap(heat_data).add_to(crash_map)

# Display the map
crash_map

In [54]:
# Create a new map centered on New York City
nyc_coords = (40.570, -74.1358)
f = folium.Figure(width=1200, height=800)
crash_map = folium.Map(location= nyc_coords, tiles="openstreetmap", zoom_start=12).add_to(f)
#crash_map = folium.Map(location=nyc_coords, zoom_start=11)

# Add a heatmap layer using the latitude and longitude columns from your DataFrame
heat_data = [[row['LATITUDE'], row['LONGITUDE']] for index, row in staten_df.iterrows()]
HeatMap(heat_data).add_to(crash_map)

# Display the map
crash_map